In [54]:
import pandas as pd
import nltk
import csv

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
#from googletrans import Translator

stemmer = SnowballStemmer('english')
#translator = Translator()
from pathlib import Path
import math
import numpy as np

'''
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
'''

from IPython.display import HTML, display


In [10]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [61]:
# Reading tsvs and getting cleaned stemmed dictionary

In [11]:
#load the given dataset into pandas for processing using only the needed columns
data = pd.read_csv('Airbnb_Texas_Rentals.csv', usecols= ['average_rate_per_night', 'bedrooms_count','city', 'date_of_listing', 'description', 'latitude','longitude','title', 'url'])


In [12]:
#create a file for every row in data(the given data of airbnb)
fileCount = 0
for index, r in data.iterrows():
    data_temp = data.loc[index:index]
    fileCount +=1
    data_temp.to_csv('data/doc_'+str(index+1)+'.tsv', sep='\t', index=False, header=False)

In [13]:
# Organizing the functions


    # list of unique words (terms) to be used to build the vocabulary
words = []

def cleanData(rawData):
        tokenizer = RegexpTokenizer(r'\w+')
        stop_words = set(stopwords.words("english"))

        # TODO translatoin
        #lang = detect(t4[0])
        #print(lang)
        #rawData = translator.translate(rawData, dest='en')

        # get words lowercased
        t0 = rawData.lower()
        # remove puctuations
        t1 = tokenizer.tokenize(t0)
        #t2Data = stemmer.stem(i for i in t1Data)
        #print(len(t1))
        # reomve stop words
        t2 =[]
        t2 = [t1[i] for i in range(0,len(t1)) if t1[i] not in stop_words]

        # stemm words
        t3 = [stemmer.stem(t2[i]) for i in range(0, len(t2))]

        # remove nummbers and strings starting with numbers
        t4 = [t3[i] for i in range(0, len(t3)) if t3[i][0].isdigit()==False]


        #print(t4)
        return(t4)

def getWordsTSV(fid):
        fileWords=[]
        for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
            #words.append(cleanData(i[0][4]))
            data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
            for w in (cleanData(data[0][4].replace('\\n', ' '))):
                if w not in fileWords: fileWords.append(w)
        return(fileWords)


In [14]:

for i in range(1,fileCount):
    t = getWordsTSV(i)
    for w in t:
        if w not in words:
            words.append(w)
#print(words)

#### PS
dunno why the stemmer is removing the last 'e' from any word
we need to look at the stem function again later

In [20]:
# writing the vocabulary file from the words 
wordsCount=0
with open('vocabulary.csv','wb') as vfile:
    for i in range(0,len(words)):
        vfile.write(str(wordsCount).encode())
        vfile.write(str('\t').encode())
        vfile.write(str(words[i]).encode())
        vfile.write('\n'.encode())
        wordsCount+=1

In [27]:
print(wordsCount)

9617


In [ ]:
# DEPRICATED
'''# creating index-file by loading vocabulary file and then comparing files with all vocabularys
with open('vocabulary.csv', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    voc = list(map(tuple, reader))
#print(voc)

index = {}
for i in range(len(voc)):
    tempL=[]

    for j in range(1,fileCount+1):
        temp = getWordsTSV(j)
        print(i)
        if voc[i][1] in temp:
            tempL.append(j)
            
        if len(tempL)!=0: index[i]= tempL
        
            
#print(index)  '''

In [55]:
# creating index-file by loading vocabulary file and then comparing files with all vocabularys
with open('vocabulary.csv', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    voc = np.array(list(map(tuple, reader)))


In [60]:
print(voc[:10])

[['0' 'welcom']
 ['1' 'stay']
 ['2' 'privat']
 ['3' 'room']
 ['4' 'queen']
 ['5' 'bed']
 ['6' 'detach']
 ['7' 'bathroom']
 ['8' 'second']
 ['9' 'floor']]


In [57]:
vocWords=voc[:,1]

In [59]:
print(len(vocWords))

9617


In [72]:
# function to get ID of a given term (this one uses the voc array instead of reading from file)
def getIDfromVoc(term):
    for t in range(len(voc)):
        if voc[t,1] == term:
            return(t)
    return(-1)

In [124]:
# unit test
getIDfromVoc('welcom')

0

In [ ]:
# unit test
print(len())

In [79]:
index = {}
for f in range(1,fileCount+1):
    temp = getWordsTSV(f)
    for t in temp:
        termID=getIDfromVoc(t)
        if termID in index:
            index[termID].append(f)
        else:
            index[termID]=[f]
                

            
#print(index)  

In [ ]:
# unit test
print(index[36])

In [84]:
# we have done another fucntion with same functionality: find above!
def getID(term):
    with open('vocabulary.csv', newline='') as voc:
        line = csv.reader(voc, delimiter='\t')
        w = list(map(tuple, line))
        #print(w[0][1])
        for i in w:
            if term == i[1]:
                return (i[0])
        else:
            return None 



In [85]:
# NO NEED FOR THIS FUNCTION function takes term ID and returns (from index) the list of files containing the term
def getDocsWithID(tid):
    for k in index:
        if tid == k:
            return(index.get(k))
    return None

#getting error: TypeError: 'int' object is not iterable

In [145]:
# fixed it to check if the doc contains all terms
def getDocswithQuery(q):
    ts = cleanData(q)
    #docsIDs = getDocsWithID(int(getID(ts[0])))
    docsIDs = index[getIDfromVoc(ts[0])]
    if docsIDs == None : return None
    if len(ts)>1:
        #docsIDs = getDocsWithID(int(getID(ts[0])))
        for t in ts[1:]:
            tid = getIDfromVoc(t)
            if tid != -1:
                tempDocs = getDocsWithID(int(tid))
                if tempDocs != None:
                    docsIDs = set(docsIDs).intersection(tempDocs)
            else: return None
    return(list(docsIDs))

In [87]:
# depricated, find below!
def old_getDetails(fid):
    with open('data/doc_'+str(fid)+'.tsv', encoding="utf8") as file:
        line = csv.reader(file, delimiter='\t')
        cls = list(map(str, *line))
        print(cls[7],cls[4], cls[2], cls[8])
        # TODO create a table and list the details


In [105]:
# funciton retuns the details of a file given file id

def getDetails(fid):
    detlist =[]
    with open('data/doc_'+str(fid)+'.tsv', encoding="utf8") as file:
        line = csv.reader(file, delimiter='\t')
        cls = list(map(str, *line))
        detlist= [cls[7],cls[4], cls[2], cls[8]]
        #TODO create a table and list the details
        return detlist

In [172]:
# I defined two functions for the visualizing stuff
def hoohleResult(r):
    rhtm ='<div class="h_result"><div class="h_title"><a href="'+r[3]+'">'+r[0]+'</a></div>'
    rhtm +='<div class="h_link"><a href="'+r[3]+'">'+r[3]+'</a></div>'
    rhtm +='<div class="h_disc">'+r[1]+'</div>'
    rhtm +='</div>'
    return (rhtm)
    
def hoohleResults(query):
    docs = getDocswithQuery(query)
    styles = open("style/style.css", "r").read()
    hhtm = '<style>%s</style>' % styles     
    hhtm +='<div class="Hoohle" ><div class="h_results">'
    if docs != None:
        for doc in docs:
            hhtm +=hoohleResult(getDetails(doc))
    else:
        hhtm +='Sorry! No results found for your request!'
    hhtm += '</div></div>'
    display(HTML(hhtm))


def printResults(query):
    docs = getDocswithQuery(query)
    #det =[['Title','Description', 'City','Url']]
    #for doc in docs:
    #    det.append(getDetails(doc))
    #columns = ('Title','Description', 'City','Url')
    #n_rows = len(det)
    html='<table align="left"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr>'
    if docs != None:

        for doc in docs:
            l = getDetails(doc)
            html+='<tr><td>'+l[0]+'</td><td>'+l[1]+'</td><td>'+l[2]+'</td><td>'+l[3]+'</td></tr>'
    else:
        html+='<tr><td>Sorry! Nothing found!</td></tr>'
    html+='</table>'
    display(HTML(html))
    #table = ff.create_table(det, height_constant=60)
    
    #plotly.offline.iplot(table, filename='simple_table')
    #plt.show()
    #return det[0][1]
    
    '''
    maybe remove the stuff we commented out if we don't need it anymore?
    '''


In [176]:
hoohleResults("Welcome to stay in private room with queen bed")

In [148]:
#create dictionary with term frequency per fid by using the modified getWordsTSV function below


def getTF(fid, term):
    tfd = {}
    words = getWordsTSV_dup(fid)
    if term in words:
        for word in words:
            if word == term:
                if word in tfd:
                    tfd[term] =  tfd[term]+1
                    #maybe replace word with term id from vocabulary
                else:
                    tfd[term] = 1
    else: return None
    for tfs in tfd:
        tfd[tfs] = tfd[tfs]/len(words)
    return tfd


#modified getWordsTSV function that creates a list with all words in the doc and does not remove the duplicates
def getWordsTSV_dup(fid):
    fileWords=[]
    for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
        #words.append(cleanData(i[0][4]))
        data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
        for w in (cleanData(data[0][4].replace('\\n', ' '))):
            fileWords.append(w)
    return(fileWords)
    

In [149]:
#calculate the inverse data frquency(idf) of a word
def getIDF(word):
    
    #get the ids of the documents in which the term occures
    files = getDocsWithID(int(getID(word)))

    #number of docs in which the terms occurs it the length of files
    #filecount = total number of files
    idf = math.log(fileCount/len(files))
    return idf
    
    

In [150]:
#calculate the tfidf for one word
def getTFIDF(fid, term):
    #getting TF for term in document with document id = fid and multiply it with the IDF of the term
    return getTF(fid, term)[term] * getIDF(term)
    ''' 
    maybe we can fuse the functions for tf, idf into the tfidf
    '''


In [151]:
getTFIDF(1,'bed')

0.1039563534909512

In [ ]:
#
def createTFIDF:
    
for key, value in d.items(): 